In [1]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [2]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# use the following two columns to train the model
features = ["Pclass", "SibSp"]

X = train_df[features]
y = train_df["Survived"]

# same columns from test dataset
X_test = test_df[features]

In [4]:
from sklearn.model_selection import train_test_split

# split into training (80%) and validation (20%) datasets
X_train, X_val, y_train, y_val = train_test_split(X, y)

In [5]:
from sklearn.preprocessing import MinMaxScaler

# put each feature in [0, 1] range

scaler  = MinMaxScaler()
X_train = scaler.fit_transform(X_train) # uses the formula (x - x_min) / (x_max - x_min)
X_val   = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [6]:
# build the neural network!

from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(units = 8, activation = "relu", input_shape = (2, )), # input layer is 2 neurons
    keras.layers.Dropout(0.2), # to help prevent overfitting
    keras.layers.Dense(units = 8, activation = "relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(units = 4, activation = "relu"),
    keras.layers.Dense(units = 1, activation = "sigmoid") # 1 unit because one output -- sigmoid since binary classification
])

2024-03-12 17:19:49.526000: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-12 17:19:49.693257: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-12 17:19:49.693294: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-12 17:19:49.713691: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-12 17:19:50.512453: W tensorflow/stream_executor/platform/de

In [18]:
learning_rate = 0.0001

model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
        loss="binary_crossentropy", 
        metrics=keras.metrics.AUC()
    )

In [21]:
history = model.fit(
            X_train, y_train, 
            epochs=500, 
            batch_size=16, 
            validation_data=(X_val, y_val)
        )

42/42 [==============================] - 0s 5ms/step - loss: 0.6146 - auc_3: 0.6718 - val_loss: 0.6240 - val_auc_3: 0.6953
Epoch 2/500
42/42 [==============================] - 0s 6ms/step - loss: 0.6117 - auc_3: 0.6784 - val_loss: 0.6239 - val_auc_3: 0.6914
Epoch 3/500
42/42 [==============================] - 0s 3ms/step - loss: 0.6092 - auc_3: 0.6862 - val_loss: 0.6241 - val_auc_3: 0.6914
Epoch 4/500
42/42 [==============================] - 0s 5ms/step - loss: 0.6139 - auc_3: 0.6716 - val_loss: 0.6242 - val_auc_3: 0.6953
Epoch 5/500
42/42 [==============================] - 0s 5ms/step - loss: 0.6080 - auc_3: 0.6895 - val_loss: 0.6241 - val_auc_3: 0.6953
Epoch 6/500
42/42 [==============================] - 0s 5ms/step - loss: 0.6077 - auc_3: 0.6921 - val_loss: 0.6239 - val_auc_3: 0.6950
Epoch 7/500
42/42 [==============================] - 0s 7ms/step - loss: 0.6119 - auc_3: 0.6731 - val_loss: 0.6241 - val_auc_3: 0.6953
Epoch 8/500
42/42 [==============================] - 0s 4ms/step - 

In [11]:
# get predictions (1s and 0s)

preds = []
for x in model.predict(X_test):
    preds.append(1 if x[0] > 0.5 else 0)

14/14 [==============================] - 0s 3ms/step


In [12]:
output = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": preds
})

In [13]:
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cba5ff19-9cb9-4e69-8275-4457214de374' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>